# 1. 简介
<img src="http://pytroll.github.io/logos/pytroll_light_small.png" width="250px"></img>
本笔记本是关于Satpy库半天教程的简介。
Satpy 是一个 Python 库，可用于处理各种地球观测卫星仪器数据，由 Pytroll 开源软件小组开发。Satpy 是多个项目的成果，包括 Pytroll mpop 和 CSPP Polar2Grid，结合各方努力构建了一个更好的项目。Satpy 库的开发始于 2015 年底，旨在帮助科学家和研究人员处理这些数据。最初的主要目标和使用案例之一是在配置的操作服务器环境中尽可能快速地生成高质量、高分辨率的图像。自创建以来的多年里，Satpy 已扩展支持更多使用案例，同时与其它开源 Python 工具的兼容性也得到了提升。Satpy 的高级功能包括：
- 读取数据文件- 合成数据产品（例如 RGB 图像）- 将数据重采样到新的地理投影- 将数据写入磁盘格式- 将数据与其他基于Python的可视化工具结合使用
本教程由多个Jupyter Notebook组成，整体上将提供对Satpy库的概述；介绍其功能以及如何使用它。在本教程中，我们将使用Satpy来读取和分析真实的卫星仪器数据。我们将介绍Satpy提供的功能范围，并将详细的科学分析作为读者的练习。
**注意**：如果您无法正确设置本地环境来运行笔记本，需要注意这些示例也可以通过 [Pangeo's BinderHub](http://pangeo-data.org) 使用 JupyterLab 界面在云平台交互式运行。请参阅此仓库的 [README](https://github.com/pytroll/tutorial-satpy-half-day/blob/main/README.rst) 文件以获取更多信息（仓库链接如下）。

## 有用的链接
* [Pytroll 主页](http://pytroll.github.io)* [Pytroll Slack 团队](http://pytroll.slack.com)* [教程仓库](https://github.com/pytroll/tutorial-satpy-half-day)* [Satpy 仓库](https://github.com/pytroll/satpy)* [Satpy 文档](https://satpy.readthedocs.io/en/latest/)

# 2. 配置
在参加本教程的讲师指导版本之前，您将在本地计算机上运行笔记本（而非在云端）必要的软件应安装并将测试数据下载到正确的位置位置。请遵循[安装](https://github.com/pytroll/tutorial-satpy-half-day/blob/main/INSTALL.md)创建正确的 conda 环境的说明。一旦完成此笔记本已在正确激活的Python环境启动后，我们可以运行以下单元格以验证安装。我们可以通过选择该单元格来运行它。并按住 'Shift' 键，然后按下 'Enter' 键。
在执行该单元格生成的输出中，你应该看到每个旁边的``ok``所列功能将在整个过程中使用本教程和返回值为 ``True``。如果否，则你的安装可能已损坏或损坏。您可能可以使用错误。消息以确定缺失或损坏的部分。对于以下内容具有 ``ok``。功能无法保证本教程的每个部分都能完成成功。

In [ ]:
from satpy.utils import check_satpy
check_satpy(readers=['abi_l1b', 'viirs_sdr'],
            writers=['geotiff', 'cf', 'simple_image'],
            extras=['cartopy', 'geoviews'])

## 下载测试数据
在开始教程之前，应已通过命令行下载测试数据。如果您尚未执行此操作，请从下方单元格中删除 `#` 并运行该单元格。如果您已执行此操作，请 **跳过** 本单元格，否则可能会重新下载数据。

In [ ]:
#%run -e ../download_data.py

# 3. 我们正在处理什么？
我们将使用地球观测气象卫星仪器。数据。数据的结构方式有多种不同的变体，它在物理上所代表的内容，随时间的变化，以及如何使用在特定类型的应用程序中。以下是一些常见气象卫星观测的特点。我们将进入几个细节将在我们稍后探讨真实数据时进行说明，并完全忽略其他为简化起见，详情如下。
## 地球观测
Satpy 处理来自地球观测卫星仪器的数据。数据可用于研究大气层、植被和海洋的变化，污染，以及其他许多因素。
<img src="../assets/GOES-16_ABI_RadF_true_color_20180910_170031_GOES-East.small.png" alt="GOES-16 ABI Full Disk True Color" width="400"/>

## 地球静止轨道与极地轨道对比
卫星可以处于高轨道地球静止轨道或较低轨道极地轨道。地球静止卫星通常提供观测数据。在相同区域以更高的时间速率更快（某些情况下约30秒），但通常具有较低的空间分辨率，并且只能看到地球的一部分。极轨卫星通常具有更高的空间分辨率并覆盖地球的更多区域，但要获得所有这些覆盖范围需要更长时间。
<table>
    <tr>
<th style="text-align:center">地球静止轨道</th><th style="text-align:center">极轨</th>    <tr>
        <td>
<video width="320" height="240" controls src="../assets/地球静止轨道.mp4"></video>        </td>
        <td>
<video width="320" height="240" controls src="../assets/polar_orbit_with_path.mp4"></video>        </td>
    </tr>
</table>

<sub>致谢：Clayton Suplinski，SSEC，威斯康星大学麦迪逊分校</sub>
## 图像
我们将要处理的仪器数据是影像数据；数据来自“成像仪”仪器。在大多数情况下，这些数据点数组可以被视为由像素组成的二维图像。

<table>
    <tr>
        <td>
            <img src="../assets/array_print_10.png" alt="Array Values" width="300"/>
        </td>
<td><font size="30px">&rarr;</font></td>        <td>
            <img src="../assets/array_imshow_10.png" alt="Array Plot" width="300"/>
        </td>
    </tr>
</table>

## 地理定位
我们将要查看的数据是地理定位数据。我们需要能够为每个分配将数据像素映射到地理区域。数据足迹可以某种程度上难以描述的内容通常通过指定来简化仅中心点和像素的半径或单元格宽度。我们可能知道每个像素的确切经度和纬度坐标或者我们可能会得到一个数据的网格化版本，其中每个像素均匀分布在矩形区域。
如果你熟悉投影，我们稍后再讨论那些。
<img src="../assets/goes16_abi_cartopy_florida.png" alt="ABI over Florida using Cartopy" width="300"/>

## 频段
我们将要处理的卫星仪器数据是一个集合辐射测量。通过进行单独测量地球上的物体反射或发射的辐射，我们可以得到一个良好的- 从太空拍摄的地球图像。卫星仪器通常具有多个频段或信道，每个频段测量特定波长电磁波谱的。每个波段都能向我们展示不同的信息关于地球。
<img src="../assets/goes16_abi_4channel_montage.png" alt="ABI C01/C04/C07/C16" width="300"/>

## 行业
一些地球静止卫星为不同领域或地区记录数据。地球的，根据其目的和设计。例如 GOES-16ABI 仪器有4个扇区：
- 全盘 (FD 或 FLDK)- 美国本土 (CONUS)- 中尺度1 (M1)- 中尺度2 (M2)
下图展示了这些不同区域以及GOES-16 ABI为它们记录数据的速度。图像中的颜色表示每个数据像素覆盖地球的面积。
<img src="../assets/goes16_sectors.png" alt="GOES-16 ABI Sectors" width="300"/>

<sub>来源：Mat Gunshor，CIMSS</sub>

# 4. 初次使用笔记本？
如果你对Jupyter Notebook没有太多经验，本节将提供一些信息，帮助你完成教程的其余部分。如果你对Notebook感到熟悉，可以自由跳到下一节。
### 细胞
笔记本由多个独立的单元格组成。本教程中的单元格有两种类型：代码和 Markdown。代码单元格包含可以由你执行的真实 Python 代码。Markdown 单元格是经过特殊格式化的单元格，用于展示说明或其它类似 HTML 的信息（如图片、视频等）。Markdown 单元格无法执行，你不需要对 Markdown 单元格做任何更改。
代码单元格可以通过选择该单元格（点击它）然后按键盘上的 `Shift` + `Enter` 来运行。这将执行该单元格并选择下一个单元格。您也可以使用 `Ctrl` + `Enter` 来执行单元格并停留在当前单元格。当代码单元格正在执行时，会在单元格左上角的括号中显示 `*`。代码单元格将按照您运行它们的顺序依次执行，不会并行运行。
### 模式
Jupyter Notebook 在使用单元格时主要有两种模式：编辑模式和命令模式。在编辑模式下，可以通过在单元格内输入内容来修改单元格的内容。可以通过点击代码单元格内部或在命令模式下按下 "Enter" 键进入编辑模式。在编辑模式下，单元格周围会出现绿色边框。
在命令模式下，您可以对单元格执行操作，例如删除单元格、在当前单元格上方或下方创建新单元格等。在命令模式下，单元格将具有蓝色边框。您可以通过按下 "Esc" 键从编辑模式进入命令模式。
### 键盘快捷键
在命令模式下，您可以使用各种键盘快捷键：
- `h`: 显示帮助和其他键盘快捷键- `a`: 在当前单元格上方创建新单元格- `b`: 在当前单元格下方创建新单元格- `dd`: 删除当前单元格
通过按下 `h` 键查看键盘快捷键窗口以获取更多信息。

# 5. 下一课
建议在学习本教程的过程中，保持主“文件浏览器”页面打开，以便轻松选择下一个笔记本。如果你已经关闭了这个页面，可以通过点击以下链接来访问它：
[**课程文件浏览器**](/tree/notebooks)

建议在完成一个课程后，通过在文件浏览器页面上选择该课程名称旁边的复选框，然后点击文件列表顶部的“Shutdown”来关闭该课程的笔记本内核。这将结束该课程的会话并删除笔记本中到目前为止的所有进度。我们在学习课程时执行此操作，以减少系统的工作量。